### Program 1
- In this assignment you will write a series of programs to solve sliding-tile puzzles using various uninformed and informed (heuristic) methods.
- **Heuristics**, or "rules of thumb," are problem-solving methods that are based on practical experience and knowledge. They allow you to use a "quick fix" to solve a minor problem or to narrow down options.
#### Part 1 – Reading and Validating Sliding-Puzzle Problems
- Read a sliding-tile puzzle problem using a JSON parser. 
- Check that the sliding-tile puzzle problem is valid. Specifically, there must be fields named *n*, *start*, and *goal*. The n field must be a positive integer greater than 1. The start and goal fields must be 𝑛 x 𝑛 matrices containing the integers 0 (for the empty space) to 𝑛2 − 1

In [2]:
import pandas as pd

df1 = pd.read_json('datasets/1-move.json')
df2 = pd.read_json('datasets/2-moves.json')
df15 = pd.read_json('datasets/15-puzzle.json')
df2


,n,start,goal
0,3,"[3, 1, 2]","[0, 1, 2]"
1,3,"[4, 0, 5]","[3, 4, 5]"
2,3,"[6, 7, 8]","[6, 7, 8]"


In [3]:
import string

def validation(df: pd.DataFrame, filename: string):
    # check if n, start and goal are defined
    if(df.columns.__contains__('n') and df.columns.__contains__('start') and df.columns.__contains__('goal')):
        # check the value of n
        if(df['n'].unique()):
            nvalue = df['n'].unique()[0]
            # if n is greater than or equal to 1, proceed
            startLength = len(df['start'])
            goalLength = len(df['goal'])
            # if the start and goal columns are of length n, proceed
            if (nvalue >= 1 and startLength == nvalue and goalLength == nvalue):
                print("'n' is greater than or equal to one. 'n' is", nvalue)
                # check the start and goal columns
                # if they are of size n AND contain a zero, proceed 
                for row,row2 in zip(df['start'],df['goal']):
                    if len(row) == nvalue and len(row2) == nvalue: 
                        if 0 in row or 0 in row2:
                            print('0 found in', row,row2)
            print(f'{filename} is valid' )
        else:
            print(f'{filename} is not valid. n is not greather than or equal to one.') 
    else:
        print(f"{filename} is not valid.JSON file does not contain fields 'n', 'start', or 'goal'")

validation(df1, '1-move.json')
validation(df2, "2-moves.json")
validation(df15, "15-puzzle.json")

'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [0, 4, 5] [3, 4, 5]
1-move.json is valid
'n' is greater than or equal to one. 'n' is 3
0 found in [3, 1, 2] [0, 1, 2]
0 found in [4, 0, 5] [3, 4, 5]
2-moves.json is valid
'n' is greater than or equal to one. 'n' is 4
0 found in [9, 0, 10, 7] [9, 10, 11, 12]
0 found in [13, 14, 11, 15] [13, 14, 15, 0]
15-puzzle.json is valid


#### Part 2 – Sliding-Tile Puzzle Rules
- Given a sliding-tile puzzle state, you must be able to determine the rules that are applicable to that state that can be used to generate its successor states.
- A rule has three parts:
    - *name* – a simple name for the rule (e.g., up, left, down, right)
    - *precondition function* – a Boolean function that accepts a state and returns true if the rule is applicable to state
    - *action function* – a function that accepts a state and returns the successor state obtained by applying the rule

In [4]:
from typing import List
from copy import deepcopy

def find_zero(state: pd.DataFrame): 
    for i, row in enumerate(state['start']):
        if 0 in row:
            return i, row.index(0)

def swap(state: pd.DataFrame, pos1, pos2):
    new_state = deepcopy(state['start'].tolist())   
    new_state[pos1[0]][pos1[1]], new_state[pos2[0]][pos2[1]] = new_state[pos2[0]][pos2[1]], new_state[pos1[0]][pos1[1]]
    return pd.DataFrame({'start': new_state})

def copy_state(state: pd.DataFrame):
    return state.copy()


def applicable_moves(state:pd.DataFrame) -> List[str]:
    # tuple = (index of row with 0, index of 0 in the row)
    zero_position = find_zero(state)
    zero_row = state['start'][zero_position[0]]
    moves = []

    # move right
    if zero_position[1] < len(zero_row) - 1:
        moves.append('Right') 
    # move left
    if zero_position[1] > 0:
        moves.append('Left') 
    # move up   
    if zero_position[0] > 0:
        moves.append('Up') 
    # move down
    if zero_position[0] < len(state) - 1: 
        moves.append('Down') 

    return moves

def move_and_apply(state: pd.DataFrame, direction: List[str], statements: bool):
    # tuple = (index of row with 0, index of 0 in the row)
    # zero index = zero_position[1]
    # zero_row_index = zero_position[0]
    zero_position = find_zero(state)
    succesor_states = []
    if statements:
        print('Starting state: \n', state)
        print()
        print('The applicable moves are: ', direction)
        print()
    for dir in direction: 

        if dir == "Right":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]
            new_zero_index = zero_position[1] + 1
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            succesor_states.append(new_state['start'].tolist())
            if(statements):
                print("Successor state after 'right'\n", new_state)
                print('\n')
        if dir == "Left":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]
            new_zero_index = zero_position[1] - 1
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            succesor_states.append(new_state['start'].tolist())
            if(statements):
                print("Successor state after 'left'\n", new_state)
                print()
        if dir == "Up":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]-1 
            new_zero_index = zero_position[1]
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            succesor_states.append(new_state['start'].tolist())
            if(statements):
                print("Successor state after 'up'\n", new_state)
                print()
        if dir == "Down":
            new_state = copy_state(state) 
            new_zero_row = zero_position[0]+1 
            new_zero_index = zero_position[1]
            new_state = swap(new_state, zero_position, (new_zero_row, new_zero_index))
            succesor_states.append(new_state['start'].tolist())
            if(statements):
                print("Successor state after 'down'\n", new_state)
                print()
    return succesor_states

def succesor_state(states:List):
    print(states)


In [5]:
applicable_moves(df2)

['Right', 'Left', 'Up', 'Down']

In [6]:
applicable_moves(df1)

['Right', 'Up', 'Down']

In [7]:
applicable_moves(df15)

['Right', 'Left', 'Up', 'Down']

In [8]:
move_and_apply(df2,applicable_moves(df2),False)

[[[3, 1, 2], [4, 5, 0], [6, 7, 8]],
 [[3, 1, 2], [0, 4, 5], [6, 7, 8]],
 [[3, 0, 2], [4, 1, 5], [6, 7, 8]],
 [[3, 1, 2], [4, 7, 5], [6, 0, 8]]]

In [9]:
move_and_apply(df1,applicable_moves(df1),True)

Starting state: 
    n      start       goal
0  3  [3, 1, 2]  [0, 1, 2]
1  3  [0, 4, 5]  [3, 4, 5]
2  3  [6, 7, 8]  [6, 7, 8]

The applicable moves are:  ['Right', 'Up', 'Down']

Successor state after 'right'
        start
0  [3, 1, 2]
1  [4, 0, 5]
2  [6, 7, 8]


Successor state after 'up'
        start
0  [0, 1, 2]
1  [3, 4, 5]
2  [6, 7, 8]

Successor state after 'down'
        start
0  [3, 1, 2]
1  [6, 4, 5]
2  [0, 7, 8]



[[[3, 1, 2], [4, 0, 5], [6, 7, 8]],
 [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
 [[3, 1, 2], [6, 4, 5], [0, 7, 8]]]

In [10]:
result = move_and_apply(df15,applicable_moves(df15),True)

Starting state: 
    n             start             goal
0  4      [1, 3, 6, 4]     [1, 2, 3, 4]
1  4     [5, 2, 8, 12]     [5, 6, 7, 8]
2  4     [9, 0, 10, 7]  [9, 10, 11, 12]
3  4  [13, 14, 11, 15]  [13, 14, 15, 0]

The applicable moves are:  ['Right', 'Left', 'Up', 'Down']

Successor state after 'right'
               start
0      [1, 3, 6, 4]
1     [5, 2, 8, 12]
2     [9, 10, 0, 7]
3  [13, 14, 11, 15]


Successor state after 'left'
               start
0      [1, 3, 6, 4]
1     [5, 2, 8, 12]
2     [0, 9, 10, 7]
3  [13, 14, 11, 15]

Successor state after 'up'
               start
0      [1, 3, 6, 4]
1     [5, 0, 8, 12]
2     [9, 2, 10, 7]
3  [13, 14, 11, 15]

Successor state after 'down'
              start
0     [1, 3, 6, 4]
1    [5, 2, 8, 12]
2   [9, 14, 10, 7]
3  [13, 0, 11, 15]



In [11]:
succesor_state(result)

[[[1, 3, 6, 4], [5, 2, 8, 12], [9, 10, 0, 7], [13, 14, 11, 15]], [[1, 3, 6, 4], [5, 2, 8, 12], [0, 9, 10, 7], [13, 14, 11, 15]], [[1, 3, 6, 4], [5, 0, 8, 12], [9, 2, 10, 7], [13, 14, 11, 15]], [[1, 3, 6, 4], [5, 2, 8, 12], [9, 14, 10, 7], [13, 0, 11, 15]]]


### Part 3
- The DATALIST argument is a list of the states in the current search path.
- The algorithm returns either a list of the rules to reach the goal state or failure if no goal was found. [You will have to decide how to represent failure in your implementation. Note that returning an empty list for failure is not a good idea because an empty list represents success when you are at the goal.]
- Implement the BACKTRACK1(DATALIST) algorithm to solve instances of the sliding-puzzle problem. The depth bound may be a global variable or passed as an argument
- Implement a main program to accept a sliding-tile puzzle problem and solve it using the BACKTRACK1(DATALIST) algorithm. Print the **start state** and the **goal state**, the **solution** and **solution length**, and the **number of states that were examined**
- Implement a main program to accept a sliding-tile puzzle problem and solve it using an *iterative depth- first search* using the BACKTRACK1(DATALIST) algorithm. Print the **cumulative number of states** examined and the **final (optimal) solution**.
- Datalist example
1. 
Start State: [[1, 2, 3], [4, 5, 6], [7, 0, 8]]

DATALIST = [start_state]

DATA ← FIRST(DATALIST) 
- DATA = [[1, 2, 3], [4, 5, 6], [7, 0, 8]]
2. 
New State: [[1, 2, 3], [4, 0, 6], [7, 5, 8]]

DATALIST = [new_state, start_state]

DATA ← FIRST(DATALIST)
- DATA = 

In [12]:
# test_state = [[7, 2, 4],[5, 0, 6],[8, 3, 1]]
# testdf = pd.DataFrame({'start': test_state})
# search(testdf,goal)


In [13]:
from collections import deque


def DFS(state:pd.DataFrame, goal):
    list_state = state['start'].tolist()

    print("start:")
    for row in list_state:
        print(" ".join(map(str, row)))
    print()
    print("goal:")
    for row in goal:
        print(" ".join(map(str, row)))
    print()

    explored = []
    stack = deque([(list_state, 0, [])]) 
    number_of_nodes = 0 
    solution_length = 0

    while stack:
        #print("stack\n", stack)
        current_state, depth, path = stack.pop()

        number_of_nodes += 1
        explored.append(tuple(map(tuple, current_state)))
        #print("explored\n", explored)

        if current_state == goal: 
            solution_length = depth
            print("Solution found!")
            print("Number_of_nodes", number_of_nodes)
            print("solution_length", solution_length)
            for move in path:
                print(f"(rule '{move}')")
            print(")")
            return

        #print("current_state", current_state)
 
        df = pd.DataFrame({'start': current_state})
        possible_states = move_and_apply(df, applicable_moves(df), False)
        #print("possible_states\n", possible_states)

        backtrack_needed = True
        moves = applicable_moves(df)
        for i in possible_states:
            i_tuple = tuple(map(tuple, i))
            #print("i\n",i)
            if i_tuple not in explored:
                stack.append((i, depth + 1, path + [current_state]))
                backtrack_needed = False
            if i == goal: 
                solution_length = depth + 1
                print("Solution found!")
                print("Number_of_nodes", number_of_nodes)
                print("solution_length", solution_length)
                return
        if backtrack_needed:
            # Backtrack: remove the current state from explored to revisit it if needed
            explored.remove(tuple(map(tuple, current_state)))

    print("No solution found.")
    return
            
                ################### BFS #########################
                # current_state = i
                # print("current_state_i", current_state)
                # df_I = pd.DataFrame({'start': i})
                # possible_states = move_and_apply(df_I, applicable_moves(df_I), False)
                # print("possible_states_i\n", possible_states)
                # if goal in possible_states:
                #     solution_length = depth + 1
                #     print("Solution found!")
                #     print("Number_of_nodes", number_of_nodes)
                #     print("solution_length", solution_length)
                #     return

def search(state: pd.DataFrame, goal):
    list_state = state['start'].tolist()
    stack = deque([(list_state, 0, [])]) 

    print("start:")
    for row in list_state:
        print(" ".join(map(str, row)))
    print()
    print("goal:")
    for row in goal:
        print(" ".join(map(str, row)))
    print()

    explored = set()
    total_nodes_generated = 0

    while stack:
        current_state, depth, path = stack.pop()  
        total_nodes_generated += 1
        
        if current_state == goal:
            print(f"Solution length = {depth}")
            print(f"Nodes examined = {total_nodes_generated}")
            print("(list")
            
            # Print each move in the solution path
            for move in path:
                print(f"(rule '{move}')")
            print(")")
            return

        # Mark current state as explored
        explored.add(tuple(map(tuple, current_state)))

        # Convert current_state back to DataFrame
        df = pd.DataFrame({'start': current_state})

        # Get the applicable moves and apply them
        result = move_and_apply(df, applicable_moves(df), False)

        # For each resulting state, push it to the stack with the move that generated it
        moves = applicable_moves(df)
        for i, descendant in enumerate(result):
            descendant_tuple = tuple(map(tuple, descendant))  # Convert lists to tuples
            if descendant_tuple not in explored and descendant not in stack:
                new_path = path + [moves[i]]  # Append the move to the path
                stack.append((descendant, depth + 1, new_path))  # Add new state, depth, and path




    

In [14]:
goal = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

search(df1,goal)

start:
3 1 2
0 4 5
6 7 8

goal:
0 1 2
3 4 5
6 7 8

Solution length = 29
Nodes examined = 30
(list
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
)


In [15]:
search(df2,goal)


start:
3 1 2
4 0 5
6 7 8

goal:
0 1 2
3 4 5
6 7 8

Solution length = 432
Nodes examined = 439
(list
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Right')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Ri

In [16]:
df3 = pd.read_json('datasets/3-moves.json')
df3

,n,start,goal
0,3,"[3, 1, 2]","[0, 1, 2]"
1,3,"[4, 7, 5]","[3, 4, 5]"
2,3,"[6, 0, 8]","[6, 7, 8]"


In [17]:
search(df3,goal)

start:
3 1 2
4 7 5
6 0 8

goal:
0 1 2
3 4 5
6 7 8

Solution length = 897
Nodes examined = 913
(list
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Down')
(rule 'Right')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Right')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Right')
(rule 'Right')
(rule 'Down')
(rule 'Down')
(rule 'Left')
(rule 'Up')
(rule 'Up')
(rule 'Left')
(rule 'Down')
(rule 'Do

In [18]:
# test_state = [[7, 2, 4],[5, 0, 6],[8, 3, 1]]
# testdf = pd.DataFrame({'start': test_state})
# DFS(testdf,goal)

In [42]:
def DFS_backtrack_iterative(state: pd.DataFrame, goal):
    """ Solve sliding-tile puzzle using iterative DFS with backtracking """

    # Convert initial state into a list for easier manipulation
    list_state = state['start'].tolist()

    # Set to track explored states and a stack to simulate DFS
    explored = set()  # Set to track explored states for fast lookup
    stack = deque([(list_state, 0, [])])  # Stack stores (current_state, depth, path)
    number_of_nodes = 0  # To count the number of states examined
    cumulative_nodes = 0

    while stack:
        # Pop the current state, depth, and path from the stack
        current_state, depth, path = stack.pop()

        # Increment the number of examined states
        number_of_nodes += 1
        current_state_tuple = tuple(map(tuple, current_state))  # Convert to tuple for hashing
        
        # If the current state matches the goal state, print the solution details and exit
        if current_state == goal:
            print("Solution found!")
            print("Cumulative number of states examined:", cumulative_nodes)
            print("Final (optimal) solution path:", path + [current_state])
            print("Solution length (depth):", depth)
            return

        # Mark the current state as explored
        explored.add(current_state_tuple)

        # Generate possible states from the current state
        df = pd.DataFrame({'start': current_state})
        possible_states = move_and_apply(df, applicable_moves(df), False)
        cumulative_nodes += len(explored)
        print("Cumulative number of states examined:", cumulative_nodes)
        # Iterate through each possible next state
        for next_state in possible_states:
            next_state_tuple = tuple(map(tuple, next_state))

            # Only add the state to the stack if it hasn't been explored
            if next_state_tuple not in explored:
                stack.append((next_state, depth + 1, path + [current_state]))

    # If no solution is found, print the number of states examined
    print("No solution found.")
    print("Cumulative number of states examined:", number_of_nodes)

In [44]:
DFS_backtrack_iterative(df1,goal)

Cumulative number of states examined: 1
Cumulative number of states examined: 3
Cumulative number of states examined: 6
Cumulative number of states examined: 10
Cumulative number of states examined: 15
Cumulative number of states examined: 21
Cumulative number of states examined: 28
Cumulative number of states examined: 36
Cumulative number of states examined: 45
Cumulative number of states examined: 55
Cumulative number of states examined: 66
Cumulative number of states examined: 78
Cumulative number of states examined: 91
Cumulative number of states examined: 105
Cumulative number of states examined: 120
Cumulative number of states examined: 136
Cumulative number of states examined: 153
Cumulative number of states examined: 171
Cumulative number of states examined: 190
Cumulative number of states examined: 210
Cumulative number of states examined: 231
Cumulative number of states examined: 253
Cumulative number of states examined: 276
Cumulative number of states examined: 300
Cumulati

In [45]:
DFS_backtrack_iterative(df2,goal)

Cumulative number of states examined: 1
Cumulative number of states examined: 3
Cumulative number of states examined: 6
Cumulative number of states examined: 10
Cumulative number of states examined: 15
Cumulative number of states examined: 21
Cumulative number of states examined: 28
Cumulative number of states examined: 36
Cumulative number of states examined: 45
Cumulative number of states examined: 55
Cumulative number of states examined: 66
Cumulative number of states examined: 78
Cumulative number of states examined: 91
Cumulative number of states examined: 105
Cumulative number of states examined: 120
Cumulative number of states examined: 136
Cumulative number of states examined: 153
Cumulative number of states examined: 171
Cumulative number of states examined: 190
Cumulative number of states examined: 210
Cumulative number of states examined: 231
Cumulative number of states examined: 253
Cumulative number of states examined: 276
Cumulative number of states examined: 300
Cumulati

In [46]:
DFS_backtrack_iterative(df3,goal)

Cumulative number of states examined: 1
Cumulative number of states examined: 3
Cumulative number of states examined: 6
Cumulative number of states examined: 10
Cumulative number of states examined: 15
Cumulative number of states examined: 21
Cumulative number of states examined: 28
Cumulative number of states examined: 36
Cumulative number of states examined: 45
Cumulative number of states examined: 55
Cumulative number of states examined: 66
Cumulative number of states examined: 78
Cumulative number of states examined: 91
Cumulative number of states examined: 105
Cumulative number of states examined: 120
Cumulative number of states examined: 136
Cumulative number of states examined: 153
Cumulative number of states examined: 171
Cumulative number of states examined: 190
Cumulative number of states examined: 210
Cumulative number of states examined: 231
Cumulative number of states examined: 253
Cumulative number of states examined: 276
Cumulative number of states examined: 300
Cumulati